(method)=
# Method

* *Present the steps and components (briefly) to create an understanding of the proposed procedure. The details can come later. Highlight what makes it unique compared to other studies in the literature (include references).*

## Overview of the proposed procedure

[Fig](overview) shows a flow chart summarizing how the proposed PIT.
```{figure} figures/method.png
---
width: 500px
name: overview
---
Flow chart over the proposed Parameter Identification Technique (PIT)
```

### VMM
The VMM (see section  [VMM](vmm) ) expresses the ship dynamics. This ability is normally used to conduct manoeuvring simulations, where a VMM with known forces from hydrodynamic derivatives is used to predict unknown ship manoeuvres. 

### Inverse dynamics
VMM can also be used to solve the reversed problem: predicting unknown forces from known ship manoeuvres, which is applicable for the current problem. This is called inverse dynamics (see section [Inverse-dynamics](inverse_dynamics)). 

### Regression
The hydrodynamic derivatives in the VMM can be identified with regression of the force polynomials on forces predicted with inverse dynamics, which is called ''motion regression'' in this paper. Many authors have used different statistical methods or machine learning methods to regress the hydrodynamic derivatives. The Linear regression with Ordinary Least Square fit (OLS) is used in the proposed PIT. The OLS is known to be sensitive to noise and outliers, which is why the main focus in the present PIT is on pre-processing data with filtering rather than the regression method itself.

### Extended Kalman Filter
The filtering in the PIT is conducted with an Extended Kalman Filter (EKF) (see section [EKF](EKF)) which is a recursive filter that estimates the state of a linear or nonlinear dynamic system from a series of noisy measurements. The basic idea is that noise can be disregarded if it does not make sense from a physical point of view. For instance if noisy measurement data would be perfectly correct, this would mean that the ship has a lot of vibrations that must have originated from huge forces, considering the large mass of the ship. The prior understanding of the model tests data suggests that these huge forces are not present during the model test, and the noise is therefore considered as measurement noise that should be removed. Lowpass filtering is a common way to do this, where motions above some cut-off frequency are regarded as unphysical measurement noise. The problem with lowpass filter is that it is hard to know what the cut-off frequency to choose, either too low: removing part of the signal, or too high: keeping some unfiltered measurement noise in the data. 
The Kalman filter has a system model that continuously estimates the state of the system that is run in parallel with the measurement data. The filter estimates the current state as a combination of the measurement data and the system model estimate based on belief in the data and the model. If the data has low measurement noise the estimate turns towards the data, if the data is instead very noisy and the model gives better predictions the estimate turns towards the model predictions instead. The inverse dynamics requires the entire state of the system to be known including: positions, velocities and accelerations. Only positions are known from the measurements in this case which means that velocities and accelerations are hidden states that can instead be estimated by the EKF.

### Rauch Tung Striebel Smoother
The EKF is recursive and can be run online, continuously making new estimates as new measurements arrive. It uses passed measurements to estimate states in the near future. This is useful for applications like autopilots. For the PIT the filter is instead run on a whole time series of existing measurements. The fact that both past and future data is known can be used to improve the filter by attaching a smoother after the EKF.  The PIT uses a Rauch Tung Striebel (RTS) smoother (see section [RTS](RTS)). 


A VMM is used as the system model in the EKF. But the identified VMM is of course not known yet which is a ''the chicken or the egg'' situation. This is solved with initial guessing and iteration.

### Step 1) 
VMM with guessed hydrodynamic derivatives is used. To make a fair guess, the derivatives are estimated with semiempirical formulas from the literature, for a linear VMM. The VMM is used in the EKF and RTS smoother to filter all the model tests. The VMM:s are assumed to have Markov property which means that future states depend only on the current state. This means that the filtered data with estimated hidden states from all the model tests can be joined into one time independent dataset that is passed to the motion regression. The hydrodynamic derivatives are regressed on quasi static forces from inverse dynamics giving the identified nonlinear VMM.

### Step 2) 
Step1 can now be rerun, but instead of using the guessed system model, the identified VMM from step 1 can now be used in the EKF. There should be a higher belief in this model than the guessed model, so the covariance matrixes should be updated.  

![](figures/method.png)

### Recursive Extended Kalman Filter

How the PIT method converges can be seen in [fig](iterations) showing simulation results with the initial guessed model and two iterrations of the PIT algorithm. 

```{glue:figure} iterations
:figwidth: 1000px
:name: "iterations"

Simulation with: initial model, first and second iteration of the PIT for one of the KVLCC2 tests

```

In [ ]:
# %load imports.py
%load_ext autoreload
%autoreload 2
%reload_kedro
%config Completer.use_jedi = False  ## (To fix autocomplete)
%matplotlib inline

import pandas as pd
pd.set_option('display.max_columns', 500)
from src.models.vmm import ModelSimulator
import matplotlib.pyplot as plt
from src.visualization.plot import track_plots, plot, captive_plot
import kedro
import numpy as np
import os.path
import anyconfig

import matplotlib
plt.style.use('paper')

from myst_nb import glue
from src.symbols import *
import src.symbols as symbols
from src.system_equations import *

from IPython.display import display, Math, Latex, Markdown
from sympy.physics.vector.printing import vpprint, vlatex

from src.parameters import df_parameters
p = df_parameters["symbol"]

# Read configs:
conf_path = os.path.join("../../conf/base/")
runs_globals_path = os.path.join(
    conf_path,
    "runs_globals.yml",
)

runs_globals = anyconfig.load(runs_globals_path)
model_test_ids = runs_globals["model_test_ids"]

join_globals_path = os.path.join(
    conf_path,
    "join_globals.yml",
)

joins = runs_globals["joins"]
join_runs_dict = anyconfig.load(join_globals_path)

globals_path = os.path.join(
    conf_path,
    "globals.yml",
)
global_variables = anyconfig.load(globals_path)



vmms = global_variables["vmms"]
from src.visualization.plot import track_plots, track_plot, plot, plot_parameters
from jb_helpers import df_to_myst, parameter_to_latex

In [ ]:
#id = '22773'
#ship = 'wpcc'
id = 'HSVA_CPMC_KVLCC2_Z_20_10_S'
ship = 'kvlcc2_hsva'

data_raw = catalog.load(f"{ship}.{id}.data")
data_lowpass = catalog.load(f"{ship}.{id}.data_lowpass")

data = catalog.load(f"{ship}.initial.{id}.data_ek_smooth")
ek = catalog.load(f"{ship}.vmm_linear.ek")
model = catalog.load(f"{ship}.initial.vmm_linear.joined.model")

vmm_name = 'vmm_abkowitz'

In [ ]:
ships = ['wpcc','kvlcc2_hsva']

ship_datas = {}
initial_parameters = {}

for ship in ships:
       
    ship_datas[ship] = catalog.load(f"{ship}.ship_data")
    initial_parameters[ship] = catalog.load(f"{ship}.initial_parameters")
    

ship_datas = pd.DataFrame(ship_datas)
ship_datas.dropna(how='any', inplace=True)

initial_parameters = pd.DataFrame(initial_parameters)
initial_parameters.drop(index='Xthrust', inplace=True)
mask = (initial_parameters!=0).all(axis=1)
initial_parameters = initial_parameters.loc[mask].copy()

In [ ]:
data['U'] = np.sqrt(data['u']**2 + data['v']**2)
df_sim = ek.simulate(data=data, input_columns=['delta','U'], solver='Radau')

dataframes = {
    'initial guessed model': df_sim,
    'PIT iteration 1': catalog.load(f"{ship}.initial.{vmm_name}.joined.{id}.data_resimulate"),
    'PIT iteration 2': catalog.load(f"{ship}.updated.{vmm_name}.joined.{id}.data_resimulate"),
    'model test' : data,
}

keys = ['u','v','r']
error_keys = [f'$\\epsilon({key})$' for key in keys]  
for key, df_ in dataframes.items():
    df_[error_keys]=df_[keys] - data[keys]

fig,ax=plt.subplots()
track_plots(dataframes, lpp=ship_datas[ship]['L'], beam=ship_datas[ship]['B'], time_window=[0,70], N=2, ax=ax);
glue("iterations",fig, display=False)

fig = plot(dataframes, keys=error_keys, ncols=1);
plt.tight_layout()
glue("iteration_errors",fig, display=False)

rmse = {}
rmse_keys = [f'$RMSe({key})$' for key in keys] 
for key, df_ in dataframes.items():
    rmse[key] = np.sqrt((df_[error_keys]**2).mean())
    

rmse = pd.DataFrame(rmse)

r = rmse.loc['$\\epsilon(r)$']
u = rmse.loc['$\\epsilon(u)$']
v = rmse.loc['$\\epsilon(v)$']
rmse.loc['total'] = u+v+r*ship_datas[ship]['L']/2